In [ ]:
#Prog projekt rövid leírása

###
#A profession.hu-ról szedtem le álláshirdetéseket
#Ebből az adatbázisból kerestem bizonyos paraméterek mentén passzoló hirdetéseket (most nektek).
#Eredetileg ebből elemzéseket szerettem volna csinálni és azt emailben kiküldeni, de ahhoz több időszakra lett volna izgalmas leszedni a hirdetéseket, amire nem volt már időm sajnos.
#A személyre szabott ajánlatokat pedig emailben küldtem ki a html kód beágyazásával.
###

In [138]:
import requests
import json
import sys
import os
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_soup(url):
    
    page = requests.get(url)
    
    if page.status_code != 200:
        return {url : page.status_code}, None
    
    soup = BeautifulSoup(page.content,'html5lib')
    
    return None, soup

In [164]:
def get_prof_list_of_jobs(base_url):
    
    dic_jobs_info = {}
    dic_of_errors = {}
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)
    
    #Range vége: 367 lesz
    for page in tqdm(range(1, 390)):
        
        res, soup = get_soup((base_url)+  "/allasok/" + str(page) + ",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3" )
        
        if res:
            continue
        
        list_of_jobs = soup.find("ul", {"class": "job-cards"}).select('li[class*="card job-card"]' )
        
        counter=0
        
        for job in list_of_jobs:
            counter+=1
            index=job['data-prof-id']
            
            dic_jobs_info[index] = {}
            dic_of_errors[index] = {}
            
            #if job.find("h2", {"class": "job-card__title"}).find("a").text.strip()!=None:
            dic_jobs_info[index]["position"] = job.find("h2", {"class": "job-card__title"}).find("a").text.strip()
            
            #if job.find("h2", {"class": "job-card__title"}).find("a")['href']!=None:
            dic_jobs_info[index]["url"] = job.find("h2", {"class": "job-card__title"}).find("a")['href']
            
            #if job.find("a", {"class": "link-icon"}).text.strip()!=None:
            dic_jobs_info[index]["company"] = job.find("a", {"class": "link-icon"}).text.strip()
            
            #if job.select('div[class*="job-card__company-address"]')[0].find("span")['title']!=None:
            dic_jobs_info[index]["address"] = job.select('div[class*="job-card__company-address"]')[0].find("span")['title']
            
            #Munkaleírás, ha van az adott álláshoz
            if job.find("div", {"class": "card-text job-card__text list_tasks"})!=None:  
                dic_jobs_info[index]["tasks"] = job.find("div", {"class": "card-text job-card__text list_tasks"}).text.strip()
            
            #Álláshirdetésen megjelenő tagek
            dic_jobs_info[index]["tags"] = job.find_all("span", {"class": "job-card__tag advertisement_tag"})    
            szamlalo=0
            for tag in dic_jobs_info[index]["tags"]:
                dic_jobs_info[index]["tags"][szamlalo]=tag.text.strip()
                szamlalo+=1
            
            #Dátum leszedése
            dic_jobs_info[index]["date"] = job.find("div", {"class": "job-card__date date"}).text.strip()
            
            #Ha mai vagy tegnapi, rendes dátumra átírni
            if dic_jobs_info[index]["date"][0:2]=="ma":
                dic_jobs_info[index]["date"]=today
            elif dic_jobs_info[index]["date"][0:2]=="te":
                dic_jobs_info[index]["date"]=yesterday
            else:
                pass
            
    
    df = pd.DataFrame(dic_jobs_info).T
    return df

In [165]:
jobs = get_prof_list_of_jobs("https://www.profession.hu/")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [166]:
jobs

,position,url,company,address,tasks,tags,date
1477516,Beágyazott hardverfejlesztő (villamosmérnök),https://www.profession.hu/allas/beagyazott-har...,PCB DESIGN Kft,"1119 Budapest, Fehérvári út 89-95.",Kapcsolási rajz tervezéseNYÁK layoutNagy kompl...,"[Villamosmérnök, Teljes munkaidő]",2020-06-24
1484329,Jelzáloghitel ügyintéző,https://www.profession.hu/allas/jelzaloghitel-...,GV Hitelközpont Kft.,"1065 Budapest, Bajcsy-Zsilinszky út 57.",Lakáshitelt igénylő ügyfelek hitelügyintézésén...,"[Pénzügyi szolgáltatások értékesítése, Teljes ...",2020-06-24
1484256,Biztonságtechnikai szerelő / technikus,https://www.profession.hu/allas/biztonsagtechn...,DEFENDER Kft,Budapest,"Beléptető-, kamera-, riasztó és egyéb gyengeár...","[Műszerész, Teljes munkaidő, céges autó, mobil...",2020-06-24
1484328,Informatika és/vagy fizika szakos tanár,https://www.profession.hu/allas/informatika-es...,FÁY ANDRÁS REFORMÁTUS ÁLTALÁNOS ISKOLA ÉS ALAP...,"2217 Gomba, Iskola utca 2.","általános iskolai tanulók nevelése-oktatása, s...","[Tanító, Tanár, Pedagógus, Teljes munkaidő, ca...",2020-06-24
1478266,Részmunkaidős recepciós-adminisztrátor,https://www.profession.hu/allas/reszmunkaidos-...,BSZJZ ÜGYVÉDI IRODA,"1036 Budapest, Bécsi út 52.",irodába érkező ügyfelek fogadása beérkező tele...,"[Recepciós, Részmunkaidő 6 óra]",2020-06-24
...,...,...,...,...,...,...,...
1475553,Mosodai munkatárs,https://www.profession.hu/allas/mosodai-munkat...,Szent Lujza Szeretetotthon,"1118 Budapest, Ménesi út 27.",A Szent Lujza Szeretetotthon azonnali belépéss...,"[Betanított munka, Teljes munkaidő]",május 27.
1475328,Csőszerelő,https://www.profession.hu/allas/csoszerelo-ter...,Terrenis Kft.,"Budapest, Pest megye",Tisztelt Munkavállalók!Dinamikusan fejlődő cég...,"[Víz- gáz- fűtésszerelő, Teljes munkaidő, éves...",május 27.
1475552,Biztonsági és körletfelügyelő,https://www.profession.hu/allas/biztonsagi-es-...,B.-A.-Z. Megyei Büntetés-végrehajtási Intézet,"3525 Miskolc, Fazekas utca 4.","Fogvatartottak őrzése, felügyelete, ellenőrzés...","[Biztonsági őr, Vagyonőr, Portás, Teljes munka...",május 27.
1475202,Hőszivattyú és klímaszerelő szakember (SA-2970),https://www.profession.hu/allas/hoszivattyu-es...,WHC Personal Személyzeti Szolgáltató Kft.,Szeged,komplett könnyűszerkezetes és hagyományos épül...,"[Egyéb szakmunka, Teljes munkaidő]",május 26.


In [167]:
jobs.to_csv('jobs_list_scrape.csv')

In [104]:
#NEM KELL
dic_jobs_details={}

for j in range(1,3):
    index=jobs.index[j-1]
    
    dic_jobs_details[index]={}
    page = requests.get(jobs["url"][j-1])
    soup = BeautifulSoup(page.content, "html.parser")

    if soup.find("div", class_=re.compile("newadv supertemplate"))==None:
        continue
            
    #Erről még nem tudjuk, hogy jó-e

    #box_content = soup.find("div", class_=re.compile("descr"))
    #Ez nem tudom, hogy minek kell.

    #dic_jobs_details[index]["requirements"] = soup.find("p", class_=re.compile("requirements")).text.strip()
    
    #Elvárások általánosan
    dic_jobs_details[index]["requirements"] = soup.find("ul", {"class": "requirements"}).text.strip()
    
    #Szükséges tapasztalat
    #dic_jobs_details[index]["experience"] = soup.find("span", {"h3": "Szükséges tapasztalat:"})
    
    dic_jobs_details[index]["experience"] = soup.find_all("ul", {"class": "offer"})



In [98]:
dic_jobs_details

{'1484325': {'requirements': 'tanári képesítés',
  'experience': [<ul class="offer">
   <ul><li>iskolánk 8 osztállyal működik</li><li>támogató, barátságos légkörű</li><li>kiemelt fizetést biztosítunk</li></ul>
   </ul>]},
 '1484327': {'requirements': '- Megbízható munkavégzés- Önállóság',
  'experience': [<ul class="offer">
   <p>- Munkaruhát biztosítunk</p><p>- Korrekt munkakörnyezet</p><p>- Megbízható tulajdonosi háttér és munkahely</p><p>- Átlagnál magasabb bérezés</p>
   </ul>]}}

In [72]:
print(soup.find("div", {"id": "experience"}))

None


In [174]:
dic_jobs_details={}

for j in tqdm(range(1,len(jobs))):
    index=jobs.index[j-1]
    
    dic_jobs_details[index]={}
    page = requests.get(jobs["url"][j-1])
    soup = BeautifulSoup(page.content, "html.parser")

    if soup.find("div", class_=re.compile("d-column__item"))==None:
        continue
    
    #Állás, munka területe
    #if soup.find("div", {"id": "orientation"})==None:
    #    continue
    try:    
        dic_jobs_details[index]["orientation"] = soup.find("div", {"id": "orientation"}).find_all("li")
        szamlalo=0
        for terulet in dic_jobs_details[index]["orientation"]:
            dic_jobs_details[index]["orientation"][szamlalo]=terulet.text.strip()
            szamlalo+=1
    except:
        continue
    
    #Szükséges tapasztalat
    try:
        dic_jobs_details[index]["experience"] = soup.find("div", {"id": "experience"}).find("li").text.strip()
    except:
        continue
        
    #Szükséges végzettség
    try:
        dic_jobs_details[index]["qualification"] = soup.find("div", {"id": "qualification"}).find("li").text.strip()
    except:
        continue
        
    #Szükséges nyelvtudás
    try:
        dic_jobs_details[index]["lang"] = soup.find("div", {"id": "lang"}).find_all("li")
        counter=0
        for nyelv in dic_jobs_details[index]["lang"]:
            dic_jobs_details[index]["lang"][counter]=nyelv.text.strip()
            counter+=1
    except:
        continue
    
    try:
        dic_jobs_details[index]["tipus"] = soup.find("div", {"id": "classification_type"}).find("li").text.strip()
    except:
        continue
    
    
df = pd.DataFrame(dic_jobs_details).T

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [172]:
page = requests.get(jobs["url"][41])
soup = BeautifulSoup(page.content, "html.parser")

#soup.find("div", class_=re.compile("d-column__item"))
soup.find("div", {"id": "orientation"})

In [175]:
df.to_csv('jobs_list_details_scrape.csv')

In [158]:
df

,orientation,experience,qualification,lang,tipus
1477516,NaN,NaN,NaN,NaN,NaN
1484329,NaN,NaN,NaN,NaN,NaN
1484256,NaN,NaN,NaN,NaN,NaN
1484328,NaN,NaN,NaN,NaN,NaN
1478266,"[Adminisztráció, Asszisztens, Irodai munka, Re...","Mindegy, vagy nem igényel tapasztalatot",Középiskola,[Angol (Alapfok/társalgási szint)],Részmunkaidő 6 óra
1479821,NaN,NaN,NaN,NaN,NaN
1481487,"[Pénzügy, Könyvelés, Könyvelés]",3-5 év szakmai tapasztalat,Középiskola,[Nem igényel nyelvtudást],Teljes munkaidő
1484311,"[Egészségügy, Gyógyszeripar, Laboráns, Gyártás...",1-3 év szakmai tapasztalat,Középiskola,[Nem igényel nyelvtudást],Teljes munkaidő
1484309,"[Adminisztráció, Asszisztens, Irodai munka, Ad...","Mindegy, vagy nem igényel tapasztalatot",Középiskola,[Angol (Középfok/kommunikációképes szint)],Teljes munkaidő
1484325,NaN,NaN,NaN,NaN,NaN


In [176]:
mergedDf = jobs.merge(df, left_index=True, right_index=True)

In [177]:
mergedDf.to_csv('jobs_list_merged_details_scrape.csv')

In [183]:
mergedDf

,position,url,company,address,tasks,tags,date,orientation,experience,qualification,lang,tipus
1477516,Beágyazott hardverfejlesztő (villamosmérnök),https://www.profession.hu/allas/beagyazott-har...,PCB DESIGN Kft,"1119 Budapest, Fehérvári út 89-95.",Kapcsolási rajz tervezéseNYÁK layoutNagy kompl...,"[Villamosmérnök, Teljes munkaidő]",2020-06-24,NaN,NaN,NaN,NaN,NaN
1484329,Jelzáloghitel ügyintéző,https://www.profession.hu/allas/jelzaloghitel-...,GV Hitelközpont Kft.,"1065 Budapest, Bajcsy-Zsilinszky út 57.",Lakáshitelt igénylő ügyfelek hitelügyintézésén...,"[Pénzügyi szolgáltatások értékesítése, Teljes ...",2020-06-24,NaN,NaN,NaN,NaN,NaN
1484256,Biztonságtechnikai szerelő / technikus,https://www.profession.hu/allas/biztonsagtechn...,DEFENDER Kft,Budapest,"Beléptető-, kamera-, riasztó és egyéb gyengeár...","[Műszerész, Teljes munkaidő, céges autó, mobil...",2020-06-24,NaN,NaN,NaN,NaN,NaN
1484328,Informatika és/vagy fizika szakos tanár,https://www.profession.hu/allas/informatika-es...,FÁY ANDRÁS REFORMÁTUS ÁLTALÁNOS ISKOLA ÉS ALAP...,"2217 Gomba, Iskola utca 2.","általános iskolai tanulók nevelése-oktatása, s...","[Tanító, Tanár, Pedagógus, Teljes munkaidő, ca...",2020-06-24,NaN,NaN,NaN,NaN,NaN
1478266,Részmunkaidős recepciós-adminisztrátor,https://www.profession.hu/allas/reszmunkaidos-...,BSZJZ ÜGYVÉDI IRODA,"1036 Budapest, Bécsi út 52.",irodába érkező ügyfelek fogadása beérkező tele...,"[Recepciós, Részmunkaidő 6 óra]",2020-06-24,"[Adminisztráció, Asszisztens, Irodai munka, Re...","Mindegy, vagy nem igényel tapasztalatot",Középiskola,[Angol (Alapfok/társalgási szint)],Részmunkaidő 6 óra
...,...,...,...,...,...,...,...,...,...,...,...,...
1475560,Autószerelő,https://www.profession.hu/allas/autoszerelo-be...,BENDE Autókontroll Kft,"1115 Budapest, Kelenföldi út 19.",Budapest 11. kerületi autójavító műhelyünkbe ö...,"[Karosszéria lakatos, Fényező, Teljes munkaidő]",május 27.,NaN,NaN,NaN,NaN,NaN
1475553,Mosodai munkatárs,https://www.profession.hu/allas/mosodai-munkat...,Szent Lujza Szeretetotthon,"1118 Budapest, Ménesi út 27.",A Szent Lujza Szeretetotthon azonnali belépéss...,"[Betanított munka, Teljes munkaidő]",május 27.,NaN,NaN,NaN,NaN,NaN
1475328,Csőszerelő,https://www.profession.hu/allas/csoszerelo-ter...,Terrenis Kft.,"Budapest, Pest megye",Tisztelt Munkavállalók!Dinamikusan fejlődő cég...,"[Víz- gáz- fűtésszerelő, Teljes munkaidő, éves...",május 27.,NaN,NaN,NaN,NaN,NaN
1475552,Biztonsági és körletfelügyelő,https://www.profession.hu/allas/biztonsagi-es-...,B.-A.-Z. Megyei Büntetés-végrehajtási Intézet,"3525 Miskolc, Fazekas utca 4.","Fogvatartottak őrzése, felügyelete, ellenőrzés...","[Biztonsági őr, Vagyonőr, Portás, Teljes munka...",május 27.,NaN,NaN,NaN,NaN,NaN


In [43]:
x=list(jobs.index)

In [45]:
jobs.index[0]

'1484325'

In [267]:
#Tailored ajánlatok
#Benya
a=mergedDf[mergedDf['position'].str.contains('Data', na=False)]
a=a[a['address'].str.contains('Budapest', na=False)]
a=a[a['experience'].str.contains('1-3', na=False)]
del a['tags']
a.columns=['Pozíció','Link címe','Vállalat neve','Cím','Feladatleírás','Hirdetés feladása','Iparág','Szükséges tapasztalat','Szükséges végzettség','Szükséges nyelvtudás','Állás típusa']
Benya=a.head(15).set_index(["Pozíció"],drop=True)
Benya

,Link címe,Vállalat neve,Cím,Feladatleírás,Hirdetés feladása,Iparág,Szükséges tapasztalat,Szükséges végzettség,Szükséges nyelvtudás,Állás típusa
Pozíció,,,,,,,,,,
Business Analyst - Reporting and Data,https://www.profession.hu/allas/business-analy...,DSZ-Services,Budapest,Support Forecasting data management and report...,2020-06-24,"[Finance, Accounting, Analyst, Advisor, Financ...",1-3 years professional experience,High school degree,[English (intermediate / communication)],Full time
Tax Master Data Specialist (VAT) (KH_100527),https://www.profession.hu/allas/tax-master-dat...,Karrier Hungária Kft.,Budapest,Ensure the standard VAT master data maintenanc...,2020-06-24,"[Finance, Accounting, Tax Advisor]",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time
Data Engineer (B74DE),https://www.profession.hu/allas/data-engineer-...,RecruIT Kft.,"Budapest, XIII. Kerület","Az adatbázisaink (Postgres, MySQL, Mongo, Redi...",2020-06-24,"[IT üzemeltetés, Telekommunikáció, Hálózati és...",1-3 év szakmai tapasztalat,Főiskola,[Angol (Középfok/kommunikációképes szint)],Teljes munkaidő
Data analyst,https://www.profession.hu/allas/data-analyst-f...,Formel D Magyarország Kft.,Budapest,Data recording in SAP Administrative support f...,2020-06-23,"[IT Operation, Telecommunication, IT Support, ...",1-3 years professional experience,High school degree,[English (intermediate / communication)],Full time
Devops Engineer ( Data Analytics and Monitoring team),https://www.profession.hu/allas/devops-enginee...,Randstad Hungary Kft.,Budapest,As a member of the Data Analytics and Monitori...,2020-06-23,"[IT Development, Programmer, Developer, Databa...",1-3 years professional experience,College degree,[English (intermediate / communication)],Full time
German speaking Master Data specialist,https://www.profession.hu/allas/german-speakin...,Randstad Hungary Kft.,Budapest,- Liaise with business process streams and Acc...,június 19.,"[Finance, Accounting, Analyst, Advisor]",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)...,Full time
Market Risk - Data and Process Support Analyst,https://www.profession.hu/allas/market-risk-da...,Morgan Stanley Magyarország Elemző Kft.,Budapest,Collaborate with stakeholders and partners dis...,június 18.,"[Bank, Insurance, Brokering, Analysis, Finance...",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time
Credit Risk Data Control Analyst,https://www.profession.hu/allas/credit-risk-da...,Morgan Stanley Magyarország Elemző Kft.,Budapest,"Understand credit risk, credit risk management...",június 18.,"[Bank, Insurance, Brokering, Analysis, Finance...",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time
Data specialist (Recognition Services),https://www.profession.hu/allas/data-specialis...,Trax Retail Ltd,"1072 Budapest, Rákóczi út 42.",Support the delivery of projects by setting up...,június 11.,"[Administration, Assistance, Office Work, Prof...",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time


In [268]:
#Tailored ajánlatok
#Hanga
y=mergedDf[mergedDf['position'].str.contains('kognitív|Data', na=False)]
y=y[y['address'].str.contains('Budapest', na=False)]
y=y[y['experience'].str.contains('1-3|Mindegy', na=False)]
del y['tags']
y.columns=['Pozíció','Link címe','Vállalat neve','Cím','Feladatleírás','Hirdetés feladása','Iparág','Szükséges tapasztalat','Szükséges végzettség','Szükséges nyelvtudás','Állás típusa']
Hanga=y.head(15).set_index(["Pozíció"],drop=True)
Hanga


,Link címe,Vállalat neve,Cím,Feladatleírás,Hirdetés feladása,Iparág,Szükséges tapasztalat,Szükséges végzettség,Szükséges nyelvtudás,Állás típusa
Pozíció,,,,,,,,,,
Logopédus / gyógypedagógus/kognitív terapeuta,https://www.profession.hu/allas/logopedus-gyog...,GEM TANULÁSI KÖZPONT KFT.,"1148 Budapest, Örs vezér tere 25.",Gyógypedagógusi vagy logopédusi feladatok ellá...,2020-06-24,"[Oktatás, Kutatás, Tudomány, Tanító, Tanár, Pe...","Mindegy, vagy nem igényel tapasztalatot",Egyetem,[Angol (Középfok/kommunikációképes szint)],Teljes munkaidő
Business Analyst - Reporting and Data,https://www.profession.hu/allas/business-analy...,DSZ-Services,Budapest,Support Forecasting data management and report...,2020-06-24,"[Finance, Accounting, Analyst, Advisor, Financ...",1-3 years professional experience,High school degree,[English (intermediate / communication)],Full time
Tax Master Data Specialist (VAT) (KH_100527),https://www.profession.hu/allas/tax-master-dat...,Karrier Hungária Kft.,Budapest,Ensure the standard VAT master data maintenanc...,2020-06-24,"[Finance, Accounting, Tax Advisor]",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time
Data Engineer (B74DE),https://www.profession.hu/allas/data-engineer-...,RecruIT Kft.,"Budapest, XIII. Kerület","Az adatbázisaink (Postgres, MySQL, Mongo, Redi...",2020-06-24,"[IT üzemeltetés, Telekommunikáció, Hálózati és...",1-3 év szakmai tapasztalat,Főiskola,[Angol (Középfok/kommunikációképes szint)],Teljes munkaidő
Data analyst,https://www.profession.hu/allas/data-analyst-f...,Formel D Magyarország Kft.,Budapest,Data recording in SAP Administrative support f...,2020-06-23,"[IT Operation, Telecommunication, IT Support, ...",1-3 years professional experience,High school degree,[English (intermediate / communication)],Full time
Devops Engineer ( Data Analytics and Monitoring team),https://www.profession.hu/allas/devops-enginee...,Randstad Hungary Kft.,Budapest,As a member of the Data Analytics and Monitori...,2020-06-23,"[IT Development, Programmer, Developer, Databa...",1-3 years professional experience,College degree,[English (intermediate / communication)],Full time
German speaking Master Data specialist,https://www.profession.hu/allas/german-speakin...,Randstad Hungary Kft.,Budapest,- Liaise with business process streams and Acc...,június 19.,"[Finance, Accounting, Analyst, Advisor]",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)...,Full time
Market Risk - Data and Process Support Analyst,https://www.profession.hu/allas/market-risk-da...,Morgan Stanley Magyarország Elemző Kft.,Budapest,Collaborate with stakeholders and partners dis...,június 18.,"[Bank, Insurance, Brokering, Analysis, Finance...",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time
Credit Risk Data Control Analyst,https://www.profession.hu/allas/credit-risk-da...,Morgan Stanley Magyarország Elemző Kft.,Budapest,"Understand credit risk, credit risk management...",június 18.,"[Bank, Insurance, Brokering, Analysis, Finance...",1-3 years professional experience,College degree,[English (higher advanced/fit for negotiation)],Full time


In [269]:
#Tailored ajánlatok
#Ádi
z=mergedDf[mergedDf['position'].str.contains('kutatás|elemz', na=False)]
z=z[z['address'].str.contains('Budapest', na=False)]
z=z[z['experience'].str.contains('1-3|Mindegy', na=False)]
del z['tags']
z.columns=['Pozíció','Link címe','Vállalat neve','Cím','Feladatleírás','Hirdetés feladása','Iparág','Szükséges tapasztalat','Szükséges végzettség','Szükséges nyelvtudás','Állás típusa']
Adi=z.head(15).set_index(["Pozíció"],drop=True)
Adi


SyntaxError: invalid syntax (<ipython-input-269-61b122ba9efc>, line 8)

In [271]:
dict_email={}
dict_email[1]={}
dict_email[2]={}
dict_email[3]={}
dict_email[1]["tablazat"]=Adi.to_html()
dict_email[1]["nev"]="Ádi"
dict_email[1]["parameter_field"]="kutatás vagy elemzés"
dict_email[1]["parameter_varos"]="Budapesten"
dict_email[1]["parameter_tapasztalat"]="1-3 év tapasztalattal végezhető"
dict_email[1]["szam"]=len(z)
dict_email[2]["tablazat"]=Hanga.to_html()
dict_email[2]["nev"]="Hanga"
dict_email[2]["parameter_field"]="data vagy kognitív"
dict_email[2]["parameter_varos"]="Budapesten"
dict_email[2]["parameter_tapasztalat"]="1-3 év tapasztalattal végezhető"
dict_email[2]["szam"]=len(y)
dict_email[3]["tablazat"]=Benya.to_html()
dict_email[3]["nev"]="Benya"
dict_email[3]["parameter_field"]="data"
dict_email[3]["parameter_varos"]="Budapesten"
dict_email[3]["parameter_tapasztalat"]="1-3 év tapasztalattal végezhető"
dict_email[3]["szam"]=len(a)

In [182]:
#Ez csak az import miatt kellett
import smtplib, ssl

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "liliprogkondrat@gmail.com"  # Enter your address
receiver_email = "kondratlilli@gmail.com"  # Enter receiver address
password = input("Type your password and press enter: ")
message = """\
Subject: Hi there

This message is sent from Python."""

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

Type your password and press enter: er*sjelsz*


In [217]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys

In [272]:
for i in range(1,4):

    smtp_server = "smtp.gmail.com"
    sender_email = "liliprogkondrat@gmail.com"  # Enter your address
    receiver_email = ["vig1396@gmail.com" ,"dormanhanga@gmail.com" ,"kbb1795@gmail.com" ] # Enter receiver address
    password = input("Type your password and press enter: ")

    msg = MIMEMultipart()
    msg['Subject'] = "Megdöbbentően személyre szabott állásajánlatok 2."
    msg['From'] = 'liliprogkondrat@gmail.com'


    html = """\
    <html>
        <p>Szia {},<br>
           Találtam pár hirdetést, ami érdekes lehet számodra. Paraméterek: {}, {}, és tartalmazza a {} szót.<br>
           Lent láthatod a legfrissebb {} hirdetést a profession.hu-ról (amennyi releváns találat van, de max 15).<br>
       
           Jó böngészést!<br>
       
           Lilkó
        </p>
    
        {}
    </html>
    """.format(dict_email[i]["nev"],dict_email[i]["parameter_varos"],dict_email[i]["parameter_tapasztalat"],dict_email[i]["parameter_field"],dict_email[i]["szam"],dict_email[i]["tablazat"])

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email[i-1], msg.as_string())


Type your password and press enter: er*sjelsz*
Type your password and press enter: er*sjelsz*
Type your password and press enter: er*sjelsz*
